# Exploration

This notebook is used for functional exploration, in order to figure out how things work, so that they then can be implemented in the allskycam.py-Module


In [ ]:
import allskycam

import cv2
import ipywidgets as widgets
from IPython.display import display

from io import BytesIO
from time import sleep
from picamera import PiCamera

from matplotlib import pyplot as plt

import time
import numpy as np

## Camera capture to stream and display single image using PiCamera

In [ ]:
stream = BytesIO()

# grab image from camera
with PiCamera() as camera:
    camera.capture(stream, format='jpeg')
    stream.seek(0)
    jpeg_image = stream.read()

# display image
widgets.Image(
    value=jpeg_image,
    format='jpg',
    width=900
)

## Camera capture to stream and display sequence of images using PiCamera

In [ ]:
stream = BytesIO()

wgt = widgets.Image(
    format='jpeg',
    width=900
)
display(wgt)

# grab image from camera
with PiCamera() as camera:
    for i in range(100):
        camera.capture(stream, format='jpeg')
        stream.seek(0)
        jpeg_image = stream.read()
        wgt.value = jpeg_image

## Acessing and controlling the camera

It is very important to set both, framerate as well as shutter_speed to consistant values.

If the frame_rate is too high, it is not possible to reach the exposure time, if the frame_rate is too low, a lot of time is needed to finalize the image capture sequence.

In this example, the frame rate time is set 1 second longer than the exposure time.

In [1]:
import picamera
from picamera import PiCamera
from time import sleep
from fractions import Fraction
import allskycam
from io import BytesIO
import cv2
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime
picamera.CAPTURE_TIMEOUT = 1000
PiCamera.CAPTURE_TIMEOUT = 1000

In [2]:
class AllSkyCamera:
    
    def __init__(self, sensor_mode=3, exposure_time=10, gain=8, resolution=(1024, 768)):
        
        framerate = 0.9/exposure_time
        self._camera = PiCamera(framerate=framerate, sensor_mode=sensor_mode)
        
        allskycam.set_analog_gain(self._camera, gain)
        allskycam.set_digital_gain(self._camera, 1)
        self._camera.resolution = resolution
        self._camera.still_stats = False
        self._camera.shutter_speed = exposure_time*1000000
        self._camera.awb_gains = (1, 1)
        self._camera.exposure_mode = 'off'
        self._camera.awb_mode = 'off'
        self._stream = BytesIO();
    
    
    @property
    def exposure_time(self):
        return self._camera.shutter_speed/1000000
    
    
    @exposure_time.setter
    def exposure_time(self, value):
        print('Setter for exposure time called with argument %f'%value)
        self._camera.framerate = min([25, 0.999/value])
        self._camera.shutter_speed = int(value*1000000)
        print('exposure time set to %f'%(self._camera.shutter_speed/1000000))
        self._camera.exposure_mode = 'off'
        self._camera.awb_mode = 'off'
    
    
    @property
    def gain(self):
        return self._camera.analog_gain
    
    
    @gain.setter
    def gain(self, value):
        print('Setter for gain called with argument %f'%value)
        allskycam.set_analog_gain(self._camera, value)
        
    
    def __del__(self):
        self._camera.close()

        
    def capture(self):
        """return raw bayer imaga data with current settings"""
        self._stream = BytesIO();
        self._camera.capture(self._stream, format='jpeg',bayer=True)
        #self.stream.seek(0)
        return allskycam.get_bayer_data_from_stream(self._stream)
    
    def save(self):
        self._stream.seek(0)
        stream2save = self._stream; #allskycam.strip_bayer_data(self._stream)
        filename = datetime.now().strftime("%y%m%d-%H%M%S.jpg")
        with open(filename, "wb") as f:
            f.write(stream2save.getbuffer())
    
    
    def auto_expose(self, raw):
        print(self.exposure_time, self.gain)
        gray = np.median(cv2.cvtColor(raw, cv2.COLOR_BAYER_GR2GRAY))
        exposure_time = 2**10/gray*self.exposure_time
        exposure_time = np.min([exposure_time, 50])
        self.exposure_time = exposure_time
        print(self.exposure_time, self.gain)

In [3]:
myCamera = AllSkyCamera()

In [4]:
myCamera.exposure_time = 0.0001
myCamera.gain = 1

Setter for exposure time called with argument 0.000100
exposure time set to 0.000214
Setter for gain called with argument 1.000000


In [ ]:
myCamera.gain = 1
while True:
    filename = datetime.now().strftime("%y%m%d-%H%M%S.jpg")
    img = myCamera.capture()
    myCamera.auto_expose(img)
    bgr = cv2.cvtColor(img, cv2.COLOR_BAYER_RG2BGR)/2**12
    del(img)
    bgr = np.clip(bgr - np.median(bgr[:100,:100,:],[0,1]),0,1)
    bgr = bgr/np.max(bgr)
    b,g,r = cv2.split(bgr)
    del(bgr)
    r = np.power(r, 0.4)*256
    g = np.power(g, 0.5)*256
    b = np.power(b, 0.3)*256
    bgr = cv2.merge([b,g,r])
    del(r,g,b)
    cv2.imwrite(filename, bgr.astype('uint16'))
    allskycam.wait_for_period(10)

Setter for gain called with argument 1.000000
0.0011 1
Setter for exposure time called with argument 0.001093
exposure time set to 0.001071
0.001071 1
0.001071 1
Setter for exposure time called with argument 0.001084
exposure time set to 0.001071
0.001071 1
0.001071 1
Setter for exposure time called with argument 0.001079
exposure time set to 0.001071
0.001071 1
0.001071 1
Setter for exposure time called with argument 0.001077
exposure time set to 0.001071
0.001071 1
0.001071 1
Setter for exposure time called with argument 0.001075
exposure time set to 0.001071
0.001071 1
0.001071 1
Setter for exposure time called with argument 0.001072
exposure time set to 0.001071
0.001071 1
0.001071 1
Setter for exposure time called with argument 0.001067
exposure time set to 0.001043
0.001043 1
0.001043 1
Setter for exposure time called with argument 0.001059
exposure time set to 0.001043
0.001043 1
0.001043 1
Setter for exposure time called with argument 0.001059
exposure time set to 0.001043
0.00

In [ ]:
rgb = cv2.cvtColor(img, cv2.COLOR_BAYER_RG2BGR)/2**12
rgb = np.clip(rgb - np.median(rgb[:100,:100,:],[0,1]),0,1)
rgb = rgb/np.max(rgb)
rgb = np.power(rgb, 0.5)*256
cv2.imwrite(filename, rgb.astype('uint16'))

In [ ]:
?np.clip

In [ ]:
!/home/pi/AllSkyCam/venv/bin/python -m pip install --upgrade pip

In [ ]:
myCamera.close_camera()

In [ ]:
plt.hist(R*1.66,512, color='r', histtype=u'step');
plt.hist(G,512, color='g', histtype=u'step');
plt.hist(B*1.52,512, color='b', histtype=u'step');
plt.xlim((0,1200))

## Looping Image Akquisition

In the next section, the image akquisition is looped. Data is fetched from the camera including raw bayer data, for evaluation, but for storing the file, raw bayer data is striped away.


In [ ]:
import picamera
from picamera import PiCamera
from time import sleep
from fractions import Fraction
picamera.CAPTURE_TIMEOUT = 240
PiCamera.CAPTURE_TIMEOUT = 240
from io import BytesIO
import time
import allskycam
import ipywidgets as widgets
from IPython.display import display

picamera.CAPTURE_TIMEOUT = 1000
PiCamera.CAPTURE_TIMEOUT = 1000

In [ ]:
stream = BytesIO()
expt = 45
period = 60

for i in range(600):
    wait_for_period(period)
    t1 = time.time()
    filename = time.strftime("%Y%m%d%H%M%S")+'.jpg'
    with PiCamera( framerate=Fraction(1, expt+1), sensor_mode=3) as camera:
        t0 = t1
        stream.seek(0)
        allskycam.set_analog_gain(camera, 8)
        allskycam.set_digital_gain(camera, 1)
        camera.resolution = (1024, 768)
        camera.still_stats = False
        camera.shutter_speed = expt*1000000
        camera.awb_gains = (2.4, 2.21)
        camera.exposure_mode = 'off'
        camera.awb_mode = 'off'
        camera.capture(stream, format='jpeg',bayer=True)
        stream.seek(0)
        jpeg_image = stream.read()
        t1 = time.time()
        if i==0:
            imw = widgets.Image(value = jpeg_image, format='jpg')
            display(imw)
        imw.value = jpeg_image
        #print('%3.2f seconds'%(t1-t0))
    stream.seek(0)
    striped = allskycam.strip_bayer_data(stream)
    with open(filename, 'wb') as f:
        f.write(striped)
        

In [ ]:
def wait_for_period(p):
    t = time.time()
    st = (int(t/p)+1)*p-t
    time.sleep(st)

In [ ]:
print(__name__)

# Save jpeg image with stripped away raw data

An image shall be akquired including raw data.
The raw data part shall be truncated away leaving behind the plain jpeg.
The truncated file without raw data shall be stored.


In [ ]:
import picamera
from picamera import PiCamera
from time import sleep
from fractions import Fraction
picamera.CAPTURE_TIMEOUT = 240
PiCamera.CAPTURE_TIMEOUT = 240
from io import BytesIO
import time
import allskycam
import ipywidgets as widgets
from IPython.display import display

picamera.CAPTURE_TIMEOUT = 1000
PiCamera.CAPTURE_TIMEOUT = 1000

stream = BytesIO()

In [ ]:
expt = 10
t0 = time.time()
with PiCamera(sensor_mode=3) as camera:
    allskycam.set_analog_gain(camera, 1)
    allskycam.set_digital_gain(camera, 1)
    camera.resolution = (1024, 768)
    camera.still_stats = False
    camera.framerate = 1/expt
    camera.shutter_speed = int(expt*1000000)
    camera.awb_gains = (2, 2.21)
    camera.exposure_mode = 'off'
    camera.awb_mode = 'off'
    camera.capture(stream, format='jpeg',bayer=True)
    t1 = time.time()
    print('%3.2f seconds for a %3.2f s exposure'%((t1-t0), camera.exposure_speed/1000000))
    print(camera.exposure_speed)
    stream.seek(0)
    jpeg_image = stream.read()

imw = widgets.Image(format='jpg')
imw.value = jpeg_image
display(imw)

In [ ]:
from picamera import PiCamera
import time

with PiCamera(sensor_mode=2) as cam:
    exposure_time = 1.0 
    cam.framerate = 1/exposure_time
    cam.iso = 100
    cam.shutter_speed = int(exposure_time*1000000)
    cam.resolution = (1600, 900)
    cam.exposure_mode = 'off'
    time.sleep(5)
    cam.capture("test.jpg")
    print(cam.exposure_speed)

# Using PiCamera[Array]-functionallity

In [ ]:
import picamera
import picamera.array

with picamera.PiCamera() as camera:
    with picamera.array.PiBayerArray(camera) as output:
        camera.capture(output, 'jpeg', bayer=True)
        print(output.array.shape)